# Loading and visualizing results ([open in colab](https://colab.research.google.com/github/openai/baselines/blob/master/docs/viz/viz.ipynb))
In order to compare performance of algorithms, we often would like to visualize learning curves (reward as a function of time steps), or some other auxiliary information about learning aggregated into a plot. Baselines repo provides tools for doing so in several different ways, depending on the goal.

## Preliminaries / TensorBoard
First, let us install baselines repo from github

For all algorithms in baselines summary data is saved into a folder defined by logger. By default, a folder $TMPDIR/openai-<date>-<time> is used; you can see the location of logger directory at the beginning of the training in the message like this:

Logging to /var/folders/mq/tgrn7bs17s1fnhlwt314b2fm0000gn/T/openai-2018-10-29-15-03-13-537078
The location can be changed by changing OPENAI_LOGDIR environment variable. For instance, 

Notice also the usage of `OPENAI_LOG_FORMAT` environment variable - this allows us to suppress printing to stdout. 
Permissible values for `OPENAI_LOG_FORMAT` environment variables are `stdout`, `log`, `csv`, and `tensorboard` (multiple values can be comma-separated). 
The latter one (`tensorboard`) dumps values in tensorboard format, which allows for their viewing in [TensorBoard](https://www.tensorflow.org/guide/summaries_and_tensorboard). TensorBoard provides nice visualization and in many cases is the easiest way to look at the summary of the results. However, if tensorboard visualization is not sufficient, read on...


## Loading results and plotting using matplotlib
Bаselines provides helper functions to load the summaries of the results as pandas dataframes. 
For instance, the following:

In [ ]:
from common import plot_util as pu
import pickle,os
#results = pu.load_results('./ppo2_results/train_logs/test_ah_withreachgoal_noctrl/numsteps_7_lr_1/obs_idx_20/b2048_no_goal_loc_8_noobs_numsteps_7_lr_1_withreachgoal_noctrl-0') 

In [ ]:
from common import plot_util as pu
import matplotlib.pyplot as plt
import numpy as np
import pickle,os
for ref_stochastic in [0,1]:
    for prsd in [7,15,16,17]:
        ho=0.99995
        if ho==0.9999:
            dmep=500
        elif ho==0.999:
            dmep=100
        elif ho==0.99995:
            dmep=150
        if ho==0.9999 and prsd==7:
            raise
        elif ho==0.9999 and prsd==17 and ref_stochastic:
            raise
        log_suf='_ho'+str(ho)+'_dmep'+str(dmep)+'_prsd'+str(prsd)
        results_summary_plots='results_summary_plots'+'_ho'+str(ho)+'_prsd'+str(prsd) #prsd 17 is poorly trained ppo reference policy
        if ref_stochastic:
            results_summary_plots+='_refsto'
        if not os.path.exists('./icra_plots_reacher/'+results_summary_plots+'/'):
            os.makedirs('./icra_plots_reacher/'+results_summary_plots+'/')
        if ref_stochastic:
            log_pre='_refsto'
            suf='_refsto'+log_suf
        else:
            log_pre=''
            suf=''+log_suf
        #suf+='_alpha1explore'
        #suf+='_model_free_different_init_collection'
        classify=True
        if classify:
            suf+='_clsf'
        for no_regress in [0,1]:
            if classify and no_regress:
                    suf+='_noreg'
            else:
                no_regress=False

            #key='min'
            #results = pu.load_results('./icra_results/reacher_train_logs'+suf+'/'); print(len(results))
            #results = pu.load_results('./icra_results/check_return_tmp/'); print(len(results))
            results = pu.load_results('./icra_results/reacher'+log_pre+'_train_logs'+log_suf+'/'); 
            if classify and no_regress:
                final_results = []
                for i in results:
                    if not 'b2048_AIP_alpha_auto-' in i.dirname:
                        final_results.append(i)
                results=final_results
            print(len(results))
            #results = pu.load_results('./icra_results/check_alpha_tmp/'); print(len(results))
            #f, axarr, g2l= pu.plot_results_alpha(results,key=key,env='Reacher-v2', average_group=True, split_fn=lambda _: '', shaded_std=False)
            f, axarr, g2l=pu.plot_results(results,env='Reacher-v2', average_group=True, split_fn=lambda _: '', shaded_std=False,aip_no_regress=no_regress)
            plt.xlabel('Number of Data', fontsize=8)
            plt.ylabel('Episode Returns', fontsize=8)
            #plt.xlabel('Episodes so far', fontsize=8)
            #plt.ylabel('Average '+key+' Alpha', fontsize=8)
            plt.tight_layout()
            plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/compare_reacher'+suf+'.png')
            plt.show()
            #results = pu.load_results('./ppo2_results/train_logs/'); print(len(results))
            #pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)
            #plt.savefig('./icra_results/c_ah_withreachgoal_ctrl1_loss.png')


        import copy
        #alpha_suf='_ho0.9999_dmep500_prsd7'
        alpha_suf=copy.copy(log_suf)
        if ref_stochastic:
            alpha_suf='_refsto'+alpha_suf
        for classify in [0,1]:
            if classify:
                keys_range=['percent']
                alpha_suf+='_clsf'
            else:
                keys_range=['min','max','mean']
            for key in keys_range:

                #results = pu.load_results('./icra_results/reacher_train_logs'+suf+'/'); print(len(results))
                #results = pu.load_results('./icra_results/check_return_tmp/'); print(len(results))
                #results = pu.load_results('./icra_results/check_alpha_tmp/'); print(len(results))
                results = pu.load_results('./icra_results/reacher'+log_pre+'_train_logs'+log_suf+'/'); 
                if classify:
                    final_results = []
                    for i in results:
                        if 'b2048_AIP_alpha_auto_clf-' in i.dirname:
                            final_results.append(i)
                    results=final_results
                else:
                    final_results = []
                    for i in results:
                        if 'b2048_AIP_alpha_auto-' in i.dirname:
                            final_results.append(i)
                    results=final_results
                print(len(results))
                f, axarr, g2l= pu.plot_results_alpha(results,key=key,env='Reacher-v2', average_group=True, split_fn=lambda _: '', shaded_std=False)
                #f, axarr, g2l=pu.plot_results(results,env='Reacher-v2', average_group=True, split_fn=lambda _: '', shaded_std=False)
                #plt.xlabel('Number of Data', fontsize=8)
                #plt.ylabel('Episode Returns', fontsize=8)
                plt.xlabel('Episodes so far', fontsize=8)
                plt.ylabel('Average '+key+' alpha', fontsize=8)
                plt.tight_layout()
                plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/reacher_check_'+key+'_alpha'+alpha_suf+'.png')
                plt.show()
                #results = pu.load_results('./ppo2_results/train_logs/'); print(len(results))
                #pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)
                #plt.savefig('./icra_results/c_ah_withreachgoal_ctrl1_loss.png')



        #folder_suf='_ho0.9999_dmep500_prsd7'
        folder_suf=copy.copy(log_suf)
        if ref_stochastic:
            folders_suf='_refsto'+folder_suf
            model_suf='_refsto_models'+folder_suf
        else:
            folders_suf=''+folder_suf
            model_suf='_models'+folder_suf
        import os
        if not os.path.exists('./icra_plots/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/'):
            os.makedirs('./icra_plots/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/')

        for classify in [0,1]:
            for sd in range(5):
                if classify:
                    alphas_save_path='./icra_results/reacher'+model_suf+'/AIP_alpha_auto_clf_seed_'+str(sd)+'_alphas'
                else:
                    alphas_save_path='./icra_results/reacher'+model_suf+'/AIP_alpha_auto_seed_'+str(sd)+'_alphas'

                with open(alphas_save_path,'rb') as pkl:
                    all_eps_min_alphas,all_eps_max_alphas,all_eps_mean_alphas, all_eps_clf_alphas_percent= pickle.load(pkl)
                plt.clf()
                if not classify:
                    plt.plot(all_eps_min_alphas,label='min_alphas')
                    plt.plot(all_eps_max_alphas,label='max_alphas')
                    plt.plot(all_eps_mean_alphas,label='mean_alphas')
                else:
                    plt.plot(all_eps_clf_alphas_percent,label='clf_alphas=1_percent')
                plt.legend()
                plt.xlabel('Episodes', fontsize=8)
                plt.ylabel('Alpha', fontsize=8)
                plt.tight_layout()
                if not classify:
                    plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/seed_'+str(sd)+'.png')
                else:
                    plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/seed_'+str(sd)+'_clf.png')
                plt.show()

In [ ]:
import copy
#alpha_suf='_ho0.9999_dmep500_prsd7'
alpha_suf=copy.copy(log_suf)
if ref_stochastic:
    alpha_suf='_refsto'+alpha_suf
for classify in [0,1]:
    if classify:
        keys_range=['percent']
        alpha_suf+='_clsf'
    else:
        keys_range=['min','max','mean']
    for key in keys_range:

        #results = pu.load_results('./icra_results/reacher_train_logs'+suf+'/'); print(len(results))
        #results = pu.load_results('./icra_results/check_return_tmp/'); print(len(results))
        #results = pu.load_results('./icra_results/check_alpha_tmp/'); print(len(results))
        results = pu.load_results('./icra_results/reacher'+log_pre+'_train_logs'+log_suf+'/'); 
        if classify:
            final_results = []
            for i in results:
                if 'b2048_AIP_alpha_auto_clf-' in i.dirname:
                    final_results.append(i)
            results=final_results
        else:
            final_results = []
            for i in results:
                if 'b2048_AIP_alpha_auto-' in i.dirname:
                    final_results.append(i)
            results=final_results
        print(len(results))
        f, axarr, g2l= pu.plot_results_alpha(results,key=key,env='Reacher-v2', average_group=True, split_fn=lambda _: '', shaded_std=False)
        #f, axarr, g2l=pu.plot_results(results,env='Reacher-v2', average_group=True, split_fn=lambda _: '', shaded_std=False)
        #plt.xlabel('Number of Data', fontsize=8)
        #plt.ylabel('Episode Returns', fontsize=8)
        plt.xlabel('Episodes so far', fontsize=8)
        plt.ylabel('Average '+key+' alpha', fontsize=8)
        plt.tight_layout()
        plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/reacher_check_'+key+'_alpha'+alpha_suf+'.png')
        #results = pu.load_results('./ppo2_results/train_logs/'); print(len(results))
        #pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)
        #plt.savefig('./icra_results/c_ah_withreachgoal_ctrl1_loss.png')

In [ ]:
#folder_suf='_ho0.9999_dmep500_prsd7'
folder_suf=copy.copy(log_suf)
if ref_stochastic:
    folders_suf='_refsto'+folder_suf
    model_suf='_refsto_models'+folder_suf
else:
    folders_suf=''+folder_suf
    model_suf='_models'+folder_suf
import os
if not os.path.exists('./icra_plots/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/'):
    os.makedirs('./icra_plots/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/')

for classify in [0,1]:
    for sd in range(5):
        if classify:
            alphas_save_path='./icra_results/reacher'+model_suf+'/AIP_alpha_auto_clf_seed_'+str(sd)+'_alphas'
        else:
            alphas_save_path='./icra_results/reacher'+model_suf+'/AIP_alpha_auto_seed_'+str(sd)+'_alphas'

        with open(alphas_save_path,'rb') as pkl:
            all_eps_min_alphas,all_eps_max_alphas,all_eps_mean_alphas, all_eps_clf_alphas_percent= pickle.load(pkl)
        plt.clf()
        if not classify:
            plt.plot(all_eps_min_alphas,label='min_alphas')
            plt.plot(all_eps_max_alphas,label='max_alphas')
            plt.plot(all_eps_mean_alphas,label='mean_alphas')
        else:
            plt.plot(all_eps_clf_alphas_percent,label='clf_alphas=1_percent')
        plt.legend()
        plt.xlabel('Episodes', fontsize=8)
        plt.ylabel('Alpha', fontsize=8)
        plt.tight_layout()
        if not classify:
            plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/seed_'+str(sd)+'.png')
        else:
            plt.savefig('./icra_plots_reacher/'+results_summary_plots+'/reacher_alpha_every_eps'+folders_suf+'/seed_'+str(sd)+'_clf.png')
        plt.show()

In [ ]:
g2l.update({'laa':234})
g2l

In [ ]:
plt.plot(-6*np.ones(100000),':',label='optimal')

will search for all folders with baselines-compatible results in ~/logs/cartpole-ppo and subfolders and return a list of `Result` objects. Each `Result` object is a named tuple with the following fields:

  * dirname: str - name of the folder from which data was loaded
  
  * metadata: dict) - dictionary with various metadata (read from metadata.json file)

  * progress: pandas.DataFrame - tabular data saved by logger as a pandas dataframe. Available if csv is in logger formats.

  * monitor: pandas.DataFrame - raw episode data (length, episode reward, timestamp). Available if environment wrapped with Monitor wrapper
  
Thus, a learning curve from a single run can be plotted as follows (note the cumulative sum trick to get convert lengths of the episodes into number of time steps taken so far)
 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
r = results[0]
#plt.plot(np.cumsum(r.monitor.l), r.monitor.r)

The raw learning curve from a single run can be very noisy. To smoothen it and analyze if any learning actually takes place, we can use `plot_util.smooth()` function:

In [ ]:
#plt.plot(np.cumsum(r.monitor.l), pu.smooth(r.monitor.r, radius=10))

Similar curve can be obtained by using logger summaries (instead of raw episode data in monitor.csv):


In [ ]:
#plt.plot(r.progress['misc/total_timesteps'], r.progress.eprewmean)
plt.plot(r.progress.eprewmean)
plt.ylabel('Average Return Over 100 Episodes')
plt.xlabel('PPO Updates')
plt.savefig('./ppo2_results/test_ah_single_loss_withreachgoal_noctrl/ah_lr_1_total_timesteps_10000000_single_seed_0_obs_idx_20_no_goal_loc_8_noobs_withreachgoal_noctrl_loss.png')

Note, however, that raw episode data is stored by the Monitor wrapper, and hence looks similar for all algorithms, whereas progress data is handled by the algorithm itself, and hence can vary (column names, type of data available) between algorithms.

## Plotting: many curves
While the loading and plotting functions described above in principle give you access to any slice of the training summaries, sometimes it is necessary to plot and compare many training runs (multiple algorithms, multiple seeds for random number generator), and usage of the functions above can get tedious and messy. For that case, `baselines.common.plot_util` provides convenience function plot_results that handles multiple `Result` objects that need to be routed in multiple plots. Consider the following bash snippet that runs ppo2 with cartpole with 6 different seeds for 30k time steps, first with rollout batch size 32, and then with batch size 128 (note that the next cell will take a little while to run):

In [ ]:
!for seed in $(seq 0 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=corl_Acrobot-v1 --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/corl_Acrobot-v1/b2048_goal_height_1.0-$seed/ --log_interval=1 --save_path=./ppo2_results/models/corl_Acrobot-v1/goal_height_1.0_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=8 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --seed=$seed; done
!for seed in $(seq 0 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=corl_Reacher-v2 --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/corl_Reacher-v2/b2048-$seed/ --log_interval=1 --save_path=./ppo2_results/models/corl_Reacher-v2/seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=8 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --seed=$seed; done

In [ ]:
!for seed in $(seq 2 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=ah --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/test_ah_withreachgoal_ctrl1/obs_idx_20/b2048_no_goal_loc_7_noobs_ctrl1-$seed/ --log_interval=1 --save_path=./ppo2_results/models/test_ah_withreachgoal_ctrl1/no_goal_loc_7_noobs_ctrl1_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=7 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --single_loss_suf=_withreachgoal_ctrl1 --ctrl_rwd=1 --seed=$seed; done
!for seed in $(seq 2 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=ah --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/test_ah_withreachgoal_ctrl1/obs_idx_20/b2048_no_goal_loc_8_noobs_ctrl1-$seed/ --log_interval=1 --save_path=./ppo2_results/models/test_ah_withreachgoal_ctrl1/no_goal_loc_8_noobs_ctrl1_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=8 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --single_loss_suf=_withreachgoal_ctrl1 --ctrl_rwd=1 --seed=$seed; done
!for seed in $(seq 2 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=ah --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/test_ah_withreachgoal/obs_idx_20/b2048_no_goal_loc_7_noobs-$seed/ --log_interval=1 --save_path=./ppo2_results/models/test_ah_withreachgoal/no_goal_loc_7_noobs_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=7 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --single_loss_suf=_withreachgoal --ctrl_rwd=0 --seed=$seed; done
!for seed in $(seq 2 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=ah --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/test_ah_withreachgoal/obs_idx_20/b2048_no_goal_loc_8_noobs-$seed/ --log_interval=1 --save_path=./ppo2_results/models/test_ah_withreachgoal/no_goal_loc_8_noobs_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=8 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --single_loss_suf=_withreachgoal --ctrl_rwd=0 --seed=$seed; done

In [ ]:
!for seed in $(seq 5 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=ah --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/test_ah_withreachgoal/obs_idx_20/b2048_no_goal_loc_7_noobs-$seed/ --log_interval=1 --save_path=./ppo2_results/models/test_ah_withreachgoal/no_goal_loc_7_noobs_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=7 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --single_loss_suf=_withreachgoal --ctrl_rwd=0 --seed=$seed; done
!for seed in $(seq 2 9); do OPENAI_LOG_FORMAT=csv python3 -m run --alg=ppo2 --env=ah --network=mlp --num_timesteps=1e6 --log_path=./ppo2_results/train_logs/test_ah_withreachgoal/obs_idx_20/b2048_no_goal_loc_8_noobs-$seed/ --log_interval=1 --save_path=./ppo2_results/models/test_ah_withreachgoal/no_goal_loc_8_noobs_seed_$seed --with_obs=0 --obs_idx=20 --ah_with_goal_loc=0 --ah_goal_loc_idx=8 --goal_height=1.0 --num_eval_eps=2 --compare=True --compare_ah_idx=7 --reacher_sd=2 --acrobot_sd=1 --need_eval=False --plot_single_loss=True --single_loss_suf=_withreachgoal --ctrl_rwd=0 --seed=$seed; done

The results of 12 runs from the cell above can be loaded just as before, via (we discard first result that is actually from the very first run in the previous section):

In [ ]:
results = pu.load_results('./ppo2_results/train_logs/test_ah_withreachgoal/obs_idx_20/'); print(len(results))
pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)
plt.savefig('./ppo2_results/c_ah_withreachgoal_loss.png')
results = pu.load_results('./ppo2_results/train_logs/test_ah_withreachgoal_ctrl1/obs_idx_20/'); print(len(results))
pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)
plt.savefig('./ppo2_results/c_ah_withreachgoal_ctrl1_loss.png')

But how do we plot all 12 of them in a sensible manner? `baselines.common.plot_util` module provides plot_results function to do just that:

The results are split into two groups based on batch size and are plotted on a separate graph. More specifically, by default plot_results considers digits after dash at the end of the directory name to be seed id and groups the runs that differ only by those together.

Showing all seeds on the same plot may be somewhat hard to comprehend and analyse. We can instead average over all seeds via the following command:

In [ ]:
results = pu.load_results('./ppo2_results/train_logs/corl_Acrobot-v1'); print(len(results))
pu.plot_results(results, average_group=True, shaded_std=False)
plt.savefig('./ppo2_results/corl_Acrobot-v1_loss.png')
results = pu.load_results('./ppo2_results/train_logs/corl_Reacher-v2'); print(len(results))
pu.plot_results(results, average_group=True,shaded_std=False)
plt.savefig('./ppo2_results/corl_Reacher-v2_loss.png')
#pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)

The lighter shade shows the standard deviation of data, and darker shade - error in estimate of the mean (that is, standard deviation divided by square root of number of seeds). Note that averaging over seeds requires resampling to a common grid, which, in turn, requires smoothing (using language of signal processing, we need to do low-pass filtering before resampling to avoid aliasing effects). You can change the amount of smoothing by adjusting `resample` and `smooth_step` arguments to achieve desired smoothing effect See the docstring of plot_util function for more info.

To plot both groups on the same graph, we can use the following:

In [ ]:
pu.plot_results(results, average_group=True, split_fn=lambda _: '')

Option `split_fn=labmda _:''` effectively disables splitting, so that all curves end up on the same panel.

Now, with many groups the overlapping shaded regions may start looking messy. We can disable either light shaded region (corresponding to standard deviation of the curves in the group) or darker shaded region (corresponding to the error in mean estimate) by using `shaded_std=False` or `shaded_err=False` options respectively. For instance,

In [ ]:
pu.plot_results(results, average_group=True, split_fn=lambda _: '', shaded_std=False)